## Final Project for the Course Applied Data Science Capstone by IBM
### Student: Fábio Vieweger Vasques

__Description__: This project has as a objective show how countries other than those with historical tradition on wine production are established around the globe.

Wine production is divided into New World and Old World. Old World is formed by countries with a long-established history of wine production, like France, Italy, Germany, Spain and Portugal. New World wines are those produced outside this traditional winegrowing area.
So, for this study, we will check the size the New World is getting, looking for vineyards around Argentina, Australia, Brasil, Canada, Chile, Mexico, New Zealand, South Africa and the United States. We will take a closer look how it is distributed aroud the globe and we will see what those vineyards has as similarities.
To be abble to accomplish this objective, we will use Foursquare API's a few times, in a loop, to locate Vineyards within those New World producers.

## About the strategie used for this notebook:
As there is a limit of daily access of foursquare API, a DB2 database has been created to store the data retrieved from Foursquare. Within this database data has been cleaned up, removing duplicities and some standards have been applied to rows which were lacking some information. All vineyards retrivied have latitude and longitude information to be plotted on a map.

### Libraries and Dependecies Needed

In [61]:
import pandas as pd # store dataframes
import requests # retrieve information from API
from pandas.io.json import json_normalize # Threat the data retrieved from the API
import ibm_db # Library to work with DB2 database
import ibm_db_dbi # Library to work DB2 with Pandas
import folium # Map library
from folium import plugins

### Database connection configuration

In [6]:
# Create database connection to persist the API data on db2 cloud
dsn_hostname = "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "ljb20609"        # e.g. "abc12345"
dsn_pwd = "nst1kws8v63^1422"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
print('Connection variables are set')

#Configure DSN
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

Connection variables are set
DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=ljb20609;PWD=nst1kws8v63^1422;


In [7]:
#Check connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  ljb20609 on host:  dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net


### Create the table where data from the API will be stored

In [ ]:
createTable = ('create table vineyards (id_forsquare varchar(50),'
                                        'vineyard_address varchar(200),'
                                        'abreviation_country varchar(5),'
                                        'city varchar(100),'
                                        'country varchar(100),'
                                        'latitude decimal(8,6),'
                                        'longitude decimal(8,6),'
                                        'city_state varchar (100),'
                                        'Vineyard_name varchar (200))')
createStmt = ibm_db.exec_immediate(conn, createTable)
createStmt
ibm_db.close

## API Connection configuration

In [10]:
client_id = 'R33E0XLKZ3PP5RI4OOQW5WDDISIYGOFCB5FBWVOMGCSDURKO'
client_secret = 'F4C0LBGKN2X4D5YETF22AREE1EM2PRGGYV222J02A13PSJF3'
categoryId='4bf58dd8d48988d1de941735' #Category for vineyards
version='20200203'

print ('ID for client is:', client_id, '\nClient secret is: ', client_secret)

ID for client is: R33E0XLKZ3PP5RI4OOQW5WDDISIYGOFCB5FBWVOMGCSDURKO 
Client secret is:  F4C0LBGKN2X4D5YETF22AREE1EM2PRGGYV222J02A13PSJF3


### Now is time to start to interact with the API and store the data within DB2 database

In [11]:
# New World Wine Countries: Argentina, Australia, Brasil, Canada, Chile, Mexico, New Zealand, South Africa and the United States.
new_world = ('Argentina', 'Australia', 'Brasil', 'Canada', 'Chile', 'Mexico', 'New Zealand', 'South Africa', 'United States')

In [ ]:
df_vineyards = pd.DataFrame()
for country in new_world:
    #set the API parameters to start the iteration
    offset = 0
    near = country
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&categoryId={}&offset={}'.format(client_id, client_secret, near, version, categoryId, offset)
    results = requests.get(url).json()
    while 'warning' not in results['response']:
        venues = results['response']['groups'][0]['items']
        df_vineyards = json_normalize(venues)
        insertVinho = ('insert into vineyards values (?, ?, ?, ?, ?, ?, ?, ?, ?)')
        prep_stmt = ibm_db.prepare(conn, insertVinho)
        for i in df_vineyards.index:
            ibm_db.bind_param(prep_stmt, 1, df_vineyards['venue.id'][i])
            ibm_db.bind_param(prep_stmt, 2, df_vineyards['venue.location.address'][i])
            ibm_db.bind_param(prep_stmt, 3, df_vineyards['venue.location.cc'][i])
            ibm_db.bind_param(prep_stmt, 4, df_vineyards['venue.location.city'][i])
            ibm_db.bind_param(prep_stmt, 5, df_vineyards['venue.location.country'][i])
            ibm_db.bind_param(prep_stmt, 6, df_vineyards['venue.location.lat'][i])
            ibm_db.bind_param(prep_stmt, 7, df_vineyards['venue.location.lng'][i])
            ibm_db.bind_param(prep_stmt, 8, df_vineyards['venue.location.state'][i])
            ibm_db.bind_param(prep_stmt, 9, df_vineyards['venue.name'][i])
            ibm_db.execute(prep_stmt)
        ibm_db.close
        offset = offset+1
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&categoryId={}&offset={}'.format(client_id, client_secret, near, version, categoryId, offset)
        results = requests.get(url).json()

## Now the data is prepared, time to extract information from it

Now let's create a dataframe from our table:

In [36]:
pconn = ibm_db_dbi.Connection(conn)
query = 'select distinct * from vineyards'

In [37]:
df_new_world = pd.read_sql(query, pconn)

Some information have strange names on address, city and state. So let's clean it up:

In [34]:
updateAddress = "update vineyards set vineyard_address = 'Not defined' where vineyard_address = '1,#QNAN000000000E+0'"
script = ibm_db.prepare(conn, updateAddress)
ibm_db.execute(script)

True

In [32]:
updateCity = "update vineyards set city = 'Not defined' where city = '1,#QNAN000000000E+0'"
script = ibm_db.prepare(conn, updateCity)
ibm_db.execute(script)

True

In [33]:
updateState = "update vineyards set city_state = 'Not defined' where city_state = '1,#QNAN000000000E+0'"
script = ibm_db.prepare(conn, updateState)
ibm_db.execute(script)

True

In [41]:
df_new_world.head()

,ID_FORSQUARE,VINEYARD_ADDRESS,ABREVIATION_COUNTRY,CITY,COUNTRY,LATITUDE,LONGITUDE,CITY_STATE,VINEYARD_NAME
0,4babc28af964a5201ec73ae3,not defined,NZ,not defined,New Zealand,-45.084868,169.180737,not defined,Akarua Winery
1,4dc723b518387d1bd5699641,not defined,NZ,Bannockburn,New Zealand,-45.039962,169.121620,Otago,Wild Earth Wines
2,549cdc88498eaaf3954b9610,not defined,NZ,not defined,New Zealand,-45.029441,168.969287,not defined,Kinross
3,4d0e7be1b4abb60cc472417a,not defined,NZ,not defined,New Zealand,-45.028898,169.197911,not defined,Jackson Q Orchards
4,4c5cb29f94fd0f478285c945,Gibbston Valley,NZ,Queenstown,New Zealand,-45.012217,168.915346,Otago,Gibbston Valley Winery


Now let's create a table named NEW_WORLD with distinct data from table VINEYARDS. This is the table we will be using, once database processing with distinct will need much more processing than without it.

In [43]:
createNewWolrd = 'create table NEW_WORLD as (select distinct * from vineyards) with data'
script = ibm_db.prepare(conn, createNewWolrd)
ibm_db.execute(script)

True

In [64]:
query = 'select * from NEW_WORLD'
df_new_world = pd.read_sql(query, pconn)
df_new_world.head()

,ID_FORSQUARE,VINEYARD_ADDRESS,ABREVIATION_COUNTRY,CITY,COUNTRY,LATITUDE,LONGITUDE,CITY_STATE,VINEYARD_NAME
0,4babc28af964a5201ec73ae3,not defined,NZ,not defined,New Zealand,-45.084868,169.180737,not defined,Akarua Winery
1,4dc723b518387d1bd5699641,not defined,NZ,Bannockburn,New Zealand,-45.039962,169.121620,Otago,Wild Earth Wines
2,549cdc88498eaaf3954b9610,not defined,NZ,not defined,New Zealand,-45.029441,168.969287,not defined,Kinross
3,4d0e7be1b4abb60cc472417a,not defined,NZ,not defined,New Zealand,-45.028898,169.197911,not defined,Jackson Q Orchards
4,4c5cb29f94fd0f478285c945,Gibbston Valley,NZ,Queenstown,New Zealand,-45.012217,168.915346,Otago,Gibbston Valley Winery


## Now that our data is ready, let's start ploting some information on a map. Let's see what we can discover.

In [72]:
# let's start again with a clean copy of the map of San Francisco
wine_map = folium.Map(location=(0,0),zoom_start = 2)
incidents = plugins.MarkerCluster().add_to(wine_map)
labels = list(df_new_world.VINEYARD_NAME)
latitudes = list(df_new_world.LATITUDE)
longitudes = list(df_new_world.LONGITUDE)


for lat, lng, name, in zip(latitudes, longitudes, labels):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=name,
    ).add_to(incidents)

wine_map

In [75]:
# New World Wine Countries: Argentina, Australia, Brasil, Canada, Chile, Mexico, New Zealand, South Africa and the United States.
old_world = ('Germany', 'France')

In [76]:
df_vineyards = pd.DataFrame()
for country in old_world:
    #set the API parameters to start the iteration
    offset = 0
    near = country
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&categoryId={}&offset={}'.format(client_id, client_secret, near, version, categoryId, offset)
    results = requests.get(url).json()
    while 'warning' not in results['response']:
        print('País:', country, 'offset:', offset)
        venues = results['response']['groups'][0]['items']
        df_vineyards = json_normalize(venues)
        insertVinho = ('insert into vineyards values (?, ?, ?, ?, ?, ?, ?, ?, ?)')
        prep_stmt = ibm_db.prepare(conn, insertVinho)
        for i in df_vineyards.index:
            ibm_db.bind_param(prep_stmt, 1, df_vineyards['venue.id'][i])
            ibm_db.bind_param(prep_stmt, 2, df_vineyards['venue.location.address'][i])
            ibm_db.bind_param(prep_stmt, 3, df_vineyards['venue.location.cc'][i])
            ibm_db.bind_param(prep_stmt, 4, df_vineyards['venue.location.city'][i])
            ibm_db.bind_param(prep_stmt, 5, df_vineyards['venue.location.country'][i])
            ibm_db.bind_param(prep_stmt, 6, df_vineyards['venue.location.lat'][i])
            ibm_db.bind_param(prep_stmt, 7, df_vineyards['venue.location.lng'][i])
            ibm_db.bind_param(prep_stmt, 8, df_vineyards['venue.location.state'][i])
            ibm_db.bind_param(prep_stmt, 9, df_vineyards['venue.name'][i])
            ibm_db.execute(prep_stmt)
        ibm_db.close
        offset = offset+1
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&categoryId={}&offset={}'.format(client_id, client_secret, near, version, categoryId, offset)
        results = requests.get(url).json()

País: Germany offset: 0
País: Germany offset: 1
País: Germany offset: 2
País: Germany offset: 3
País: Germany offset: 4
País: Germany offset: 5
País: Germany offset: 6
País: Germany offset: 7
País: Germany offset: 8
País: Germany offset: 9
País: Germany offset: 10
País: Germany offset: 11
País: Germany offset: 12
País: Germany offset: 13
País: Germany offset: 14
País: Germany offset: 15
País: Germany offset: 16
País: Germany offset: 17
País: Germany offset: 18
País: Germany offset: 19
País: Germany offset: 20
País: Germany offset: 21
País: Germany offset: 22
País: Germany offset: 23
País: Germany offset: 24
País: Germany offset: 25
País: Germany offset: 26
País: Germany offset: 27
País: Germany offset: 28
País: Germany offset: 29
País: Germany offset: 30
País: Germany offset: 31
País: Germany offset: 32
País: Germany offset: 33
País: Germany offset: 34
País: Germany offset: 35
País: Germany offset: 36
País: Germany offset: 37
País: Germany offset: 38
País: Germany offset: 39
País: Germ